In [55]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from transformers import AutoTokenizer, AutoModel, AdamW, pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import KernelPCA
from matplotlib import pyplot as plt
from tqdm import tqdm
from typing import Optional, Tuple, List
import itertools
import hashlib
import io
import os
import glob

In [26]:
files = glob.glob("../data/truths/ncp*")

In [34]:
data = list()
for _file in files:
    text = open(_file).read()
    data.append(text)

In [38]:
test = data[0].split('\n')
test

['PATIENT NO: - ST DAVIDS MEDICAL CTR BILLING DATE PAGE 2 06676',
 'MED REC NO: 919 E. 32ND STREET 12/29/17',
 'GUARANTOR NO:',
 'PATIENT: AUSTIN TX 787052703 ADMITTED DISCHARGED',
 'CALVER DREU 04/02/17 04/05/17',
 'DATE OF BATCH F NDC/CPT-4/',
 'SERVICE REF DEPT S PROC HCPCS QTY SERVICE DESCRIPTION CHARGES',
 '040417 04B039 0712 440161 904404073 1 ASPIRIN 81MG CHEW TAB 1.00',
 '040417 04B133 0712 443838 904629261 2 ATORVASTATIN 40 MG TAB 18.00',
 '040417 04B039+ 0712 444089 62584026511 2 METOPROLOL 25MG TAB 8.00',
 'D40517 05B260 0712 445365 186077739 1 TICAGRELOR 90 MG TAB 46.00',
 '040517 05B260 0712 444089 62584026511 1 METOPROLOL 25MG TAB 4,00',
 'D40517 05B260 0712 440161 63739043401 1 ASPIRIN 81MG CHEW TAB 1.00',
 'SUBTOTAL: 515.29',
 '260-IV THERAPY',
 '040217 O6PARC 0780 780328 96374 1 IVP SINGLE/INITIAL DRU 275.00',
 'SUBTOTAL : 275.00',
 '272-STERILE SUPPLIES',
 '040217 03B882+ 0718 319311 2 SET ADM IV 125IN 20 GT 122 .00',
 '040317 03B881 0718 281479 1 BAND TR BND 24CM REG

In [50]:
test_pipeline = pipeline("feature-extraction", model="bert-base-uncased", tokenizer="bert-base-uncased", 
                                            padding=True,
                                            truncation=True,
                                            max_length=512,
                                            return_tensors='pt'
)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [51]:
test_pipeline.encode = lambda x: np.array([_[0][0] for _ in list(map(test_pipeline,x))])

In [52]:
cluster_embs = test_pipeline.encode(test)

In [56]:
from pydoc import doc


def generate_truths(
    document_repo: Optional["str"],
    output_dir: Optional["str"],
) -> pd.DataFrame:
    dfs: List = list()
    document_repo: str = f"{document_repo}/.*txt"
    files = glob.glob(document_repo)
    get_lines: List  = lambda _: (
        open(_).read().split('\n'), os.path.basename(_).split(".")[0]
    )
    documents  = list(map(get_lines, files))
    for document in documents:
        df = pd.DataFrame({
            'output': document[0],
            'document_tag': document[1],
        })
    
    

In [ ]:
generate_truths("data/truths", output_dir=None)